In [ ]:
result = adfuller(site_df)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])

In [ ]:
data_diff = site_df.diff().dropna()

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
plot_acf(data_diff, ax=ax1)
plot_pacf(data_diff, ax=ax2)
plt.show()

In [ ]:

p, d, q = 3, 0, 1
P, D, Q, s = 1, 1, 1, 24

model = SARIMAX(site_df, order=(p, d, q), seasonal_order=(P, D, Q, s))

results = model.fit(disp=False)

print(results.summary())


forecast = results.get_forecast(steps=100)
forecast_df = forecast.summary_frame(alpha=0.05)


plt.figure(figsize=(15, 7))
plt.plot(site_df, label='Original Data')
plt.plot(forecast_df['mean'], color='red', label='Forecast')
plt.fill_between(forecast_df.index, forecast_df['mean_ci_lower'], forecast_df['mean_ci_upper'], color='pink', alpha=0.3, label='95% Confidence Interval')
plt.title('SARIMA Forecast of NO2 Levels')
plt.xlabel('Date Time')
plt.ylabel('NO2 Concentration (µg/m³)')
plt.legend()
plt.grid(True)
plt.show()

results.plot_diagnostics(figsize=(15, 12))
plt.show()